In [1]:
# set the root path
import os
os.chdir('/Users/chuan/Desktop/fsan830/fsan830spring2025')
import pandas as pd

# Prediction data

In [2]:

df_0426 = pd.read_csv("data/processed/CDX0426_processed.csv")
df_0515 = pd.read_csv("data/processed/CDX0515_processed.csv")
# remove columns with all NaN values
df_0426 = df_0426.dropna(axis=1, how='all')
df_0515 = df_0515.dropna(axis=1, how='all')
# remove columns start with name col_
df_0426 = df_0426.loc[:, ~df_0426.columns.str.startswith('col_')]
df_0515 = df_0515.loc[:, ~df_0515.columns.str.startswith('col_')]
df_0426.shape, df_0515.shape

((86, 197), (86, 197))

In [3]:
# remove columns with NaN values
df_0426 = df_0426.dropna(axis=1, how='any')
df_0515 = df_0515.dropna(axis=1, how='any')
df_0426.shape, df_0515.shape

((86, 50), (86, 50))

# Training Data

In [4]:
ds_path1 = "students/he_chenchuan/data/train/processed_race_data.nc"
ds_path2 = "students/he_chenchuan/data/train/processed_race_data_with_results.nc"

import xarray as xr
ds1 = xr.open_dataset(ds_path1)
ds2 = xr.open_dataset(ds_path2)

In [6]:
var_names_ds1 = ds1.variables
var_names_ds2 = ds2.variables
# find the common variables
common_vars = set(var_names_ds1) & set(var_names_ds2)
# find the variables in ds1 but not in ds2
vars_only_in_ds1 = set(var_names_ds1) - set(var_names_ds2)
# find the variables in ds2 but not in ds1
vars_only_in_ds2 = set(var_names_ds2) - set(var_names_ds1)
print(vars_only_in_ds2)

{'scratched', 'finish_position'}


In [7]:
ds1

<xarray.Dataset> Size: 2MB
Dimensions:                        (race: 179, starter: 23, past_race: 5)
Coordinates:
  * race                           (race) object 1kB 'CD-05-02-23-R01' ... 'C...
  * starter                        (starter) int32 92B 0 1 2 3 4 ... 19 20 21 22
  * past_race                      (past_race) int32 20B 0 1 2 3 4
Data variables: (12/21)
    horse                          (race, starter) object 33kB ...
    jockey                         (race, starter) object 33kB ...
    trainer                        (race, starter) object 33kB ...
    recent_trainer                 (race, starter, past_race) object 165kB ...
    recent_race_id                 (race, starter, past_race) object 165kB ...
    recent_jockey                  (race, starter, past_race) object 165kB ...
    ...                             ...
    recent_num_starters            (race, starter, past_race) int16 41kB ...
    program_number                 (race, starter) object 33kB ...
    odds                           (race, starter) float32 16kB ...
    surface                        (race) object 1kB ...
    distance_f                     (race) float64 1kB ...
    purse                          (race) float64 1kB ...

In [8]:
# convert ds2 to a pandas dataframe, with key be race and starter, for dimension past_race indexed variables
#  add _past_race_index to the variable name
df_ds2 = ds2.to_dataframe(
    dim_order=['race', 'starter', 'past_race']
)
df_ds2 = df_ds2.reset_index()
df_ds2[:3]

,race,starter,past_race,horse,jockey,trainer,recent_trainer,recent_race_id,recent_jockey,recent_date,...,recent_purse,recent_start_pos,recent_num_starters,program_number,odds,finish_position,scratched,surface,distance_f,purse
0,CD-05-02-23-R01,0,0,Gormleyesque,Rodriguez,Catalano,Catalano,KEE-2023-04-00-R03,Peterson,2023-04-19,...,43000.0,1,11,1,2.6,2,False,D,7.0,30000.0
1,CD-05-02-23-R01,0,1,Gormleyesque,Rodriguez,Catalano,Catalano,TP-2023-03-00-R06,Loveberry,2023-03-25,...,70000.0,1,11,1,2.6,2,False,D,7.0,30000.0
2,CD-05-02-23-R01,0,2,Gormleyesque,Rodriguez,Catalano,Catalano,TP-2023-02-00-R05,Mojica,2023-02-15,...,70000.0,7,12,1,2.6,2,False,D,7.0,30000.0


In [9]:
# remove all columns start with recent_
df_ds2 = df_ds2.loc[:, ~df_ds2.columns.str.startswith('recent_')]
df_ds2[:3]

,race,starter,past_race,horse,jockey,trainer,program_number,odds,finish_position,scratched,surface,distance_f,purse
0,CD-05-02-23-R01,0,0,Gormleyesque,Rodriguez,Catalano,1,2.6,2,False,D,7.0,30000.0
1,CD-05-02-23-R01,0,1,Gormleyesque,Rodriguez,Catalano,1,2.6,2,False,D,7.0,30000.0
2,CD-05-02-23-R01,0,2,Gormleyesque,Rodriguez,Catalano,1,2.6,2,False,D,7.0,30000.0


In [10]:
# drop all rows with past_race != 0, and remove past_race column
df_ds2 = df_ds2[df_ds2['past_race'] != 0]
df_ds2 = df_ds2.drop(columns=['past_race'])
df_ds2


,race,starter,horse,jockey,trainer,program_number,odds,finish_position,scratched,surface,distance_f,purse
1,CD-05-02-23-R01,0,Gormleyesque,Rodriguez,Catalano,1,2.6,2,False,D,7.0,30000.0
2,CD-05-02-23-R01,0,Gormleyesque,Rodriguez,Catalano,1,2.6,2,False,D,7.0,30000.0
3,CD-05-02-23-R01,0,Gormleyesque,Rodriguez,Catalano,1,2.6,2,False,D,7.0,30000.0
4,CD-05-02-23-R01,0,Gormleyesque,Rodriguez,Catalano,1,2.6,2,False,D,7.0,30000.0
6,CD-05-02-23-R01,1,Kentucky Reign,Navas,Colebrook,2,9.0,3,False,D,7.0,30000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
20579,CD-05-29-23-R09,21,,,,,NaN,0,False,T,9.0,120000.0
20581,CD-05-29-23-R09,22,,,,,NaN,0,False,T,9.0,120000.0
20582,CD-05-29-23-R09,22,,,,,NaN,0,False,T,9.0,120000.0
20583,CD-05-29-23-R09,22,,,,,NaN,0,False,T,9.0,120000.0


In [11]:
# for each race, only keep remove duplicate by starter, only keep the first one
df_ds2 = df_ds2.drop_duplicates(subset=['race', 'starter'])
df_ds2

,race,starter,horse,jockey,trainer,program_number,odds,finish_position,scratched,surface,distance_f,purse
1,CD-05-02-23-R01,0,Gormleyesque,Rodriguez,Catalano,1,2.6,2,False,D,7.0,30000.0
6,CD-05-02-23-R01,1,Kentucky Reign,Navas,Colebrook,2,9.0,3,False,D,7.0,30000.0
11,CD-05-02-23-R01,2,Toto'srubyslippers,Loveberry,Sims,3,7.0,0,True,D,7.0,30000.0
16,CD-05-02-23-R01,3,Dogwoodsmilliejane,Gilligan,Taylor,4,13.0,1,False,D,7.0,30000.0
21,CD-05-02-23-R01,4,Girls House,Saez,Cano,5,2.2,4,False,D,7.0,30000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
20561,CD-05-29-23-R09,18,,,,,NaN,0,False,T,9.0,120000.0
20566,CD-05-29-23-R09,19,,,,,NaN,0,False,T,9.0,120000.0
20571,CD-05-29-23-R09,20,,,,,NaN,0,False,T,9.0,120000.0
20576,CD-05-29-23-R09,21,,,,,NaN,0,False,T,9.0,120000.0


In [12]:
df_ds2.columns

Index(['race', 'starter', 'horse', 'jockey', 'trainer', 'program_number',
       'odds', 'finish_position', 'scratched', 'surface', 'distance_f',
       'purse'],
      dtype='object')

In [13]:
map = {
  "track_code": "race", 
  "race_number": "race", 
  "program_number": "program_number",
  "odds": "odds",
  "horse_name": "horse",
  # "jockey": "jockey",
  "trainer_name": "trainer",
  "purse": "purse",
  "surface_code": "surface",
  "distance": "distance_f"
}
pred_cols = list(map.keys())
train_cols = list(map.values())

In [14]:
pred_cols

['track_code',
 'race_number',
 'program_number',
 'odds',
 'horse_name',
 'trainer_name',
 'purse',
 'surface_code',
 'distance']

In [15]:
# key only values in map
df_ds2[train_cols]

,race,race,program_number,odds,horse,trainer,purse,surface,distance_f
1,CD-05-02-23-R01,CD-05-02-23-R01,1,2.6,Gormleyesque,Catalano,30000.0,D,7.0
6,CD-05-02-23-R01,CD-05-02-23-R01,2,9.0,Kentucky Reign,Colebrook,30000.0,D,7.0
11,CD-05-02-23-R01,CD-05-02-23-R01,3,7.0,Toto'srubyslippers,Sims,30000.0,D,7.0
16,CD-05-02-23-R01,CD-05-02-23-R01,4,13.0,Dogwoodsmilliejane,Taylor,30000.0,D,7.0
21,CD-05-02-23-R01,CD-05-02-23-R01,5,2.2,Girls House,Cano,30000.0,D,7.0
...,...,...,...,...,...,...,...,...,...
20561,CD-05-29-23-R09,CD-05-29-23-R09,,NaN,,,120000.0,T,9.0
20566,CD-05-29-23-R09,CD-05-29-23-R09,,NaN,,,120000.0,T,9.0
20571,CD-05-29-23-R09,CD-05-29-23-R09,,NaN,,,120000.0,T,9.0
20576,CD-05-29-23-R09,CD-05-29-23-R09,,NaN,,,120000.0,T,9.0


In [16]:
# keep only the keys in map
df_0426[pred_cols]

,track_code,race_number,program_number,odds,horse_name,trainer_name,purse,surface_code,distance
0,CD,1,1,3.5,BALLADRY,PITTS HELEN,60000,D,1430
1,CD,1,2,2.5,WHERE'S THE WINE,CATALANO WAYNE M,60000,D,1430
2,CD,1,3,3.0,PRINCESS POM POM,FORSTER GRANT T,60000,D,1430
3,CD,1,4,6.0,ASK AMANDA,BAHENA ISMAEL,60000,D,1430
4,CD,1,5,2.0,SPIRIT RULES,SANTAMARIA CARLOS,60000,D,1430
...,...,...,...,...,...,...,...,...,...
81,CD,8,10,5.0,NYQUIST FREQUENCY,BLAIR JORDAN,67000,D,1540
82,CD,8,11,15.0,ELECTIONEERING,DEVAUX CHERIE,67000,D,1540
83,CD,8,12,3.5,TAPAKENA,LYNCH BRIAN A,67000,D,1540
84,CD,8,13,10.0,MO JACKSON,TOMLINSON MICHAEL A,67000,D,1540


In [17]:
# check any horse name matched between df_0426 and df_ds2
horse_names_0426 = [i.lower() for i in df_0426.horse_name.unique()]
horse_names_ds2 = [i.lower() for i in df_ds2.horse.unique()]
# check if any horse name in horse_names_0426 is in horse_names_ds2
intersect_horse_names = set(horse_names_0426) & set(horse_names_ds2)
intersect_horse_names

set()

In [11]:
# also track for trainer
trainers_0426 = [i.lower() for i in df_0426.trainer_name]
trainers_names_ds2 = [i.lower() for i in df_ds2.trainer.unique()]
# check if any horse name in horse_names_0426 is in horse_names_ds2
# all to lower case
intersect_trainer_names = set(trainers_0426) & set(trainers_names_ds2)
intersect_trainer_names

set()